<a href="https://colab.research.google.com/github/a-forty-two/cylons/blob/master/07_Deep_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
# https://archive.ics.uci.edu/ml/datasets/auto+mpg
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from tensorflow import keras

TensorFlow 1.x selected.
1.15.2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
filepath = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
data = keras.utils.get_file('auto-mpg.data', filepath)
print(data)
!ls '/root/.keras/datasets/'
# wget also -> we could download public data (only on linux command line)

/root/.keras/datasets/auto-mpg.data
auto-mpg.data


multi-valued discrete -> Categorical

continuous -> can be used for regression 

NaN, NaT -> Not a number, not a time 

pd.drop_na takes care of all types of NA 


In [10]:
# !cat '/root/.keras/datasets/auto-mpg.data'
col_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'model_year', 'origin', 'car_name']
# now we are aware that these are TSV (tab-sep) and not simple CSVs
fl = pd.read_csv(data, names=col_names, sep=' ', comment='\t', na_values='?', skipinitialspace=True)
fl.head()
fl['car_name']
data = fl.iloc[:,:-1] # skipping car names are this is useless data for us (CORRELATION TEST, HEATMAP)
data.head() 


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [11]:
data.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64

In [12]:
# REPLACE MISSING VALUES -> cloud computing in week 3
# Read cleaning modes on https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/clean-missing-data
# Dropping them is another way
df = data.dropna()
df.isna().sum()



mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64

In [14]:
df.head(20)

# Y is MPG , rest Cylinders to origin is X 
df.origin.value_counts()
# Assume -> origin was city name converted to numbers (ENCODING)
# 1-> BOM, 2-> BLR, 3-> MAA 
# WE are trying to calculate MILES-PER-GALLON 


1    245
3     79
2     68
Name: origin, dtype: int64

In [15]:
df.head()
# mpg = w1*cylinders + w2*displacement + w3*hp ... does this mean that model_year and origin 
# were mathematical in nature? ABSOLUTELY NOT!!! 

# WHENVER INDEPENDET CATEGORIES are present -> Origin is BLR independent of origin in BOM
# INDEPENDE, DISTINCT VALUES
# In such a scenario -> DOES it mean -> 
# y = w * city + bias 
# does it mean that if city_blr was 1, and city_bom was 2, then BOM is twice of BLR?
# THIS IS NOT A VALID RELATIONSHIP! 
# IN such a scenrio-> we apply One-Hot-Encoding 
# this encoding creates NEW columns for every city -> SUCH that the city that is useful
# is set to 1, and others to 0 
# BOM MAA BLR
#   0   1   0    -> this means that w1*BOM + w2*MAA + w3*BLR -> w1*0 + w2*MAA + w3*0
# hence only Chennai's weight would have survived! OTHER CITIES' weights would have become 0!
# thus giving a ML equation much better suited for individual city-level prediction 





,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
